In [1]:
!pip install transformers torch geopy folium sentencepiece

In [2]:
from transformers import pipeline
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import folium
from time import sleep

In [3]:
geolocator = Nominatim(user_agent="crisislens_hack")

In [4]:
classifier = pipeline(
    "sentiment-analysis",
    model="cardiffnlp/twitter-roberta-base-sentiment-latest",
    return_all_scores=True
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cpu
/usr/local/lib/python3.12/dist-packages/transformers/pipelines/text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [5]:
reports = [
    {"text": "House collapsed on Gandhi Street! My brother is trapped!! #ChennaiFlood", "source": "Twitter"},
    {"text": "No electricity or water in Adyar. Elderly couple needs insulin.", "source": "Telegram"},
    {"text": "Relief camp at Teynampet School — running low on food and blankets.", "source": "News"},
    {"text": "Road to Kottivakkam blocked by fallen trees. Ambulance can't pass.", "source": "Twitter"}
]

In [6]:
def get_urgency_score(text):
    scores = classifier(text)[0]
    try:
        neg_score = next(s['score'] for s in scores if s['label'] == 'negative')
    except StopIteration:
        neg_score = min(s['score'] for s in scores)
    return neg_score

def geocode_location(text):
    locations = [
        "Chennai", "Gandhi Street, Chennai", "Adyar, Chennai",
        "Teynampet, Chennai", "Kottivakkam, Chennai"
    ]
    for loc in locations:
        if loc.split(',')[0].lower() in text.lower():
            try:
                return geolocator.geocode(loc, timeout=10)
            except GeocoderTimedOut:
                continue
    return geolocator.geocode("Chennai, India")

In [7]:
processed = []
for r in reports:
    urgency = get_urgency_score(r["text"])
    loc = geocode_location(r["text"])

    life_threat_keywords = ["trapped", "buried", "collapsed", "bleeding", "unconscious", "crushed"]
    if any(kw in r["text"].lower() for kw in life_threat_keywords):
        urgency = max(urgency, 0.95)

    if loc:
        if urgency > 0.9:
            color = "red"
        elif urgency > 0.6:
            color = "orange"
        else:
            color = "green"

        processed.append({
            "text": r["text"],
            "urgency": urgency,
            "lat": loc.latitude,
            "lon": loc.longitude,
            "color": color
        })
    sleep(1)


In [8]:
processed

[{'text': 'House collapsed on Gandhi Street! My brother is trapped!! #ChennaiFlood',
  'urgency': 0.95,
  'lat': 13.0836939,
  'lon': 80.270186,
  'color': 'red'},
 {'text': 'No electricity or water in Adyar. Elderly couple needs insulin.',
  'urgency': 0.8200251460075378,
  'lat': 13.00645,
  'lon': 80.2577791,
  'color': 'orange'},
 {'text': 'Relief camp at Teynampet School — running low on food and blankets.',
  'urgency': 0.5617715120315552,
  'lat': 13.0443237,
  'lon': 80.2498455,
  'color': 'green'},
 {'text': "Road to Kottivakkam blocked by fallen trees. Ambulance can't pass.",
  'urgency': 0.7419747114181519,
  'lat': 12.9705053,
  'lon': 80.2615033,
  'color': 'orange'}]

In [9]:
m = folium.Map(location=[13.0827, 80.2707], zoom_start=12)

# Add pins
for item in processed:
    folium.CircleMarker(
        location=[item["lat"], item["lon"]],
        radius=10,
        popup=f"<b>{item['color'].upper()} • Urgency: {item['urgency']:.2f}</b><br><i>→ {'Keyword boost' if item['urgency'] >= 0.95 else 'Model score'}</i><br>{item['text']}",
        color=item["color"],
        fill=True,
        fill_color=item["color"]
    ).add_to(m)

In [10]:
m